# NLU Final Project 
## Understanding Indirect Answers Using Circa

Fine-Tune BERT on MNLI, and then test performance on Circa dataset 

### Install Packages

In [ ]:
# ! pip install datasets, transformers
!pip install transformers

     |████████████████████████████████| 2.2MB 6.0MB/s 
     |████████████████████████████████| 870kB 23.7MB/s 
     |████████████████████████████████| 3.3MB 30.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=3e80358d675c7115d8b54c9f2bbacc44eae1c2bd9063e55f31daf5c3f42b8f7e
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
!pip install datasets

     |████████████████████████████████| 194kB 5.4MB/s 
     |████████████████████████████████| 112kB 9.6MB/s 
     |████████████████████████████████| 245kB 7.2MB/s 


### Fine-tuning BERT on MNLI (and GLUE)

In [ ]:
# GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

GLUE_TASKS = ["mnli", "mnli-mm"]

In [ ]:
task = "mnli"
model_checkpoint = "distilbert-base-uncased"

### Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
# load circa dataset 
dataset_circa = load_dataset("circa")

Using custom data configuration default


Dataset circa downloaded and prepared to /root/.cache/huggingface/datasets/circa/default/1.1.0/63a3d813ccf4539d111770b6ff986c4572301eaef5e2095f02e3554e26ac404f. Subsequent calls will reuse this data.


In [ ]:
dataset_circa

DatasetDict({
    train: Dataset({
        features: ['context', 'question-X', 'canquestion-X', 'answer-Y', 'judgements', 'goldstandard1', 'goldstandard2'],
        num_rows: 34268
    })
})

In [ ]:
dataset["train"][0]

{'hypothesis': 'Product and geography are what make cream skimming work. ',
 'idx': 0,
 'label': 1,
 'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.'}

In [ ]:
dataset_circa["train"][0]

{'answer-Y': "I'm a veterinary technician.",
 'canquestion-X': 'I am employed .',
 'context': 'Y has just travelled from a different city to meet X.',
 'goldstandard1': 0,
 'goldstandard2': 0,
 'judgements': 'Yes#Yes#Yes#Yes#Yes',
 'question-X': 'Are you employed?'}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,hypothesis,idx,label,premise
0,"Light or dark green, madam?",304252,neutral,Light or dark green?
1,"I enjoyed visiting with you, but have to run.",153294,entailment,well it i guess we've talked probably long enough nice talking to you too i enjoyed it bye-bye
2,Nowhere recycles as much as we do in Palo Alto.,258270,entailment,uh-huh well i used to i lived there for a long time but i'm now in Palo Alto California which is i think the recycling capital of the world
3,Allowing uncertainty to drive an investor out is one of the worst things they can do.,63455,entailment,Time tells investors not to panic about the bad world One of the worst things [investors] could do is let rising volatility and uncertainty drive them out of stock investments.
4,Tourists flock to Paris in order to protest the government policies.,169475,contradiction,"Paris and its immediate surroundings are a magnet for tourists, students, businessmen, artists, inventors ' in short, everyone except perhaps the farmer and fisherman, who may well come to the city to protest government policies."
5,You need to get real,158068,entailment,Get real!
6,Users are very glad that their feedback is being heard and it is important.,335967,neutral,"Prototyping which a preliminary version of part or all of the hardware or software is developed to permit user feedback, determine feasibility, or investigate timing or other issues in support of the development process."
7,Estrogen therapy is a good option for married couples.,135729,neutral,"add two years if you're a happily married man, and subtract three if you don't exercise), while a sidebar explains the many benefits and few risks of estrogen therapy."
8,I just remembered that you don't approve of it.,216306,entailment,I forgot you had a prejudice against it.
9,There are rockets at the Aerosece Complex.,158374,entailment,"At the Aerosece Complex there are rockets, satellites and an impressive IMAX Theater ."


In [ ]:
show_random_elements(dataset_circa["train"])

,answer-Y,canquestion-X,context,goldstandard1,goldstandard2,judgements,question-X
0,There are two or three I dislike.,I get along with our colleagues .,X and Y are colleagues who are leaving work on a Friday at the same time.,"Yes, subject to some conditions","Yes, subject to some conditions","Yes, subject to some conditions#No#Yes, subject to some conditions#Yes, subject to some conditions#In the middle, neither yes nor no",Do you get along with our colleagues?
1,My parents moved to Costa Rica.,My parents live in the neighborhood .,X and Y are childhood neighbours who unexpectedly run into each other at a cafe.,No,No,No#No#No#No#No,Do your parents still live in the neighborhood?
2,I have a tech job I'll start in the fall.,I have a job lined up in New York .,Y has just told X that he/she is thinking of buying a flat in New York.,Yes,Yes,Yes#Yes#Yes#Yes#Yes,Do you have a job lined up in New York?
3,It's always been a dream of mine.,I have been thinking of moving to New York for a long time .,Y has just told X that he/she is thinking of buying a flat in New York.,Yes,Yes,Yes#Yes#Yes#Yes#Yes,Have you been thinking of moving to New York for a long time?
4,I work with Amanda.,I have seen anyone else from the old neighborhood .,X and Y are childhood neighbours who unexpectedly run into each other at a cafe.,Yes,Yes,I am not sure how X will interpret Y’s answer#Yes#I am not sure how X will interpret Y’s answer#Yes#Yes,Have you seen anyone else from the old neighborhood?
5,almost everyone.,I get along with everyone on this street .,Y has just moved into a neighbourhood and meets his/her new neighbour X.,I am not sure how X will interpret Y’s answer,Other,"Probably yes / sometimes yes#Yes, subject to some conditions#Probably yes / sometimes yes#No#Yes, subject to some conditions",Do you get along with everyone on this street?
6,Depends how expensive the tickets are.,I want to go to the Drake concert in town .,X wants to know about Y's music preferences.,"Yes, subject to some conditions","Yes, subject to some conditions","Yes, subject to some conditions#Yes, subject to some conditions#Yes, subject to some conditions#Yes, subject to some conditions#Yes, subject to some conditions",Want to go to the Drake concert in town?
7,I rode the rollercoaster over and over!,I have been to the new adventure park .,X wants to know what activities Y likes to do during weekends.,Yes,Yes,Yes#Yes#Yes#Yes#Yes,Have you been to the new adventure park?
8,I would move here if I could.,I think 'll visit more often .,Y has just travelled from a different city to meet X.,I am not sure how X will interpret Y’s answer,Yes,"Yes#Probably yes / sometimes yes#Probably yes / sometimes yes#Yes#In the middle, neither yes nor no",Think you'll visit more often?
9,I read every night before bed.,I read alot .,X wants to know what sorts of books Y likes to read.,Yes,Yes,Yes#Yes#Yes#Yes#Yes,Do you read alot?


In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.453125}

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Use 🤗 Transformers `Tokenizer`:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
    "circa": ("question-X","answer-Y" )
}

We can double check it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


In [ ]:
# to tokenize circa: 
task_c = "circa"
sentence1_key_c, sentence2_key_c = task_to_keys[task_c]
if sentence2_key_c is None:
    print(f"Sentence: {dataset_circa['train'][0][sentence1_key_c]}")
else:
    print(f"Sentence 1: {dataset_circa['train'][0][sentence1_key_c]}")
    print(f"Sentence 2: {dataset_circa['train'][0][sentence2_key_c]}")

Sentence 1: Are you employed?
Sentence 2: I'm a veterinary technician.


In [ ]:
task_c = "circa"
sentence1_key_c, sentence2_key_c = task_to_keys[task_c]
sentence1_key_c

'question-X'

In [ ]:
task_to_keys[task_c]

('question-X', 'answer-Y')

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
def preprocess_function_c(examples):
    if sentence2_key_c is None:
        return tokenizer(examples[sentence1_key_c], truncation=True)
        
    return tokenizer(examples[sentence1_key_c], examples[sentence2_key_c], truncation=True)

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 17158, 2135, 6949, 8301, 25057, 2038, 2048, 3937, 9646, 1011, 4031, 1998, 10505, 1012, 102, 4031, 1998, 10505, 2024, 2054, 2191, 6949, 8301, 25057, 2147, 1012, 102], [101, 2017, 2113, 2076, 1996, 2161, 1998, 1045, 3984, 2012, 2012, 2115, 2504, 7910, 2017, 4558, 2068, 2000, 1996, 2279, 2504, 2065, 2065, 2027, 5630, 2000, 9131, 1996, 1996, 6687, 2136, 1996, 13980, 5630, 2000, 2655, 2000, 9131, 1037, 3124, 2013, 6420, 1037, 2059, 1037, 3313, 1037, 3124, 3632, 2039, 2000, 5672, 2032, 1998, 1037, 2309, 1037, 3124, 3632, 2039, 2000, 5672, 2032, 102, 2017, 4558, 1996, 2477, 2000, 1996, 2206, 2504, 2065, 1996, 2111, 9131, 1012, 102], [101, 2028, 1997, 2256, 2193, 2097, 4287, 2041, 2115, 8128, 3371, 2135, 1012, 102, 1037, 2266, 1997, 2026, 2136, 2097, 15389, 2115, 4449, 2007, 14269, 11718, 1012, 102], [101, 2129, 2079, 2017, 2113, 1029, 2035, 2023, 2003, 2037, 2592, 2153, 1012, 102, 2023, 2592, 7460, 2000, 2068, 1012, 102], [101, 3398, 1045, 2425, 2017, 2054, 2295, 2065, 20

In [ ]:
# split circa dataset into train/test/validate

# The full `train` split.
# train_ds = datasets.load_dataset('bookcorpus', split='train')

# The full `train` split and the full `test` split as two distinct datasets.
# train_ds, test_ds, val_ds = datasets.load_dataset('circa', split=['train', 'test', 'val'])
# train_ds,test_ds, val_ds = map(datasets.load_dataset('circa', ('train', 'test[:25%]+test[50%:75%]', 'test[75%:]')))
# # The full `train` and `test` splits, concatenated together.
# train_test_ds = datasets.load_dataset('circa', split='train+test')
train_ds, test_ds, val_ds = datasets.load_dataset('circa', split=['train[:60%]', 'train[60%:80%]', 'train[80%:]'])
# train_ds_c= datasets.load_dataset('circa', split=['train[:60%]+train[60%:80%]+train[80%:]'])

Using custom data configuration default
Reusing dataset circa (/root/.cache/huggingface/datasets/circa/default/1.1.0/63a3d813ccf4539d111770b6ff986c4572301eaef5e2095f02e3554e26ac404f)


In [ ]:
test_ds

Dataset({
    features: ['context', 'question-X', 'canquestion-X', 'answer-Y', 'judgements', 'goldstandard1', 'goldstandard2'],
    num_rows: 6853
})

In [ ]:
val_ds

Dataset({
    features: ['context', 'question-X', 'canquestion-X', 'answer-Y', 'judgements', 'goldstandard1', 'goldstandard2'],
    num_rows: 6854
})

In [ ]:
train_ds

Dataset({
    features: ['context', 'question-X', 'canquestion-X', 'answer-Y', 'judgements', 'goldstandard1', 'goldstandard2'],
    num_rows: 20561
})

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9847
    })
})

In [ ]:
# encoded_dataset_c = dataset_c.map(preprocess_function_c, batched=True)

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# mnli has 3 labels: neutral, contradiction, entailment
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
# will be using accuracy for mnli
batch_size = 16
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

# mnli: learning rate = 2e-5, no epochs = 3, batch size = 16 

In [ ]:
# compute prediction metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
# validation from dataset dict 
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [38]:
# finetune model using train
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
# check if we reloaded the best model 
trainer.evaluate()

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [ ]:
! pip install optuna
! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()